In [1]:
# torch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import transforms
# read videos
import pylab          # play video
from os import listdir
import cv2
# others
import os
from PIL import Image
import random
import matplotlib.pyplot as plt
# SINDy
import sys
sys.path.append("src")
import sindy_utils as sindy
import numpy as np


### Functions

In [2]:
# print progress, possibly faster than progress bar
def printProgress(epoch, batch_id, accuracy, loss):
    """
    print progress of the training
    epoch: number
    batch_id: current batch_id
    accuracy:
    loss:
    
    """
    progress = '='* int((10. * (batch_id+1) / len(train_data)))
    progress += '>'
    if batch_id == 0:
        print('Train Epoche {}: {}% [{}]\t accuracy: {:.6f}, loss: {:.6f}'.format(
            epoch+1, int(100. * (batch_id+1) / len(train_data)),progress, accuracy, loss.item()), end='')
    else:
        print('\rTrain Epoche {}: {}% [{}]\t accuracy: {:.6f}, loss: {:.6f}'.format(
            epoch+1, int(100. * (batch_id+1) / len(train_data)),progress, accuracy, loss.item()), end='', flush = True)
        

def calculateLoss(network, params):
    """
    calculate the loss of autoencoder and SINDy combined. loss function of:
    
     O \   _________           ________  /  O
     .    |         | \  O  / |        |    .
     . -  | phi'(x) | -  O  - | phi(z) | -  .
     .    |_________| /  O  \ |________|    .
     O /                                 \  O
    x(t)                z(t)               xa(t)
    
    ||x-phi(z)||_2^2 + lam1 ||dx - (zGrad phi(z)) Theta(z^T) Xi||_2^2 + lam2 ||dz - Theta(z^T) Xi||_2^2 + lam3 ||Xi||_1
        decoder      +                   SINDy in dx                  +         SINDy in dz             +   SINDy sparsity
     
    dz = xGrad phi'(x) dx = (xGrad z) dx
    
    network: data of the network
    params: hyperparameters
    
    """
    dx = network['dx']
    dx_decode = network['dx_decode']
    dz = network['dz']
    dz_predict = network['dz_predict']
    Xi_coeff = network['Xi']
    rec_loss = network['rec_loss']
    sindy_x_loss = torch.mean((dx - dx_decode)**2)
    sindy_z_loss = torch.mean((dz - dz_predict)**2)
    sparse_loss = torch.mean(torch.abs(Xi_coeff))
    
    # separate view of each loss
    separate_loss = []
    separate_loss.append(rec_loss)
    separate_loss.append(sindy_x_loss)
    separate_loss.append(sindy_z_loss)
    separate_loss.append(sparse_loss)
    
    tot_loss = (params['loss_weight_decoder'] * rec_loss
                + params['loss_weight_sindy_x'] * sindy_x_loss 
                + params['loss_weight_sindy_z'] * sindy_z_loss
                + params['loss_weight_sindy_regularization'] * sparse_loss)
                                                                                        
    return tot_loss, separate_loss


def calculateSindy(network, params):
    '''
    Calculate Theta(z), Xi and dz
    
    '''
    z = network['z'].detach().numpy()
    dz = network['dz'].detach().numpy()
    
    network['Theta'] = torch.from_numpy(sindy.sindy_library(z, params['poly_order'], include_sine=params['include_sine']))
    network['Xi'] = torch.from_numpy(sindy.sindy_fit(network['Theta'], dz, params['sindy_threshold']))
    dz_predict = np.dot(network['Theta'],network['Xi'])
    
    return torch.from_numpy(dz_predict)

### Define model parameters

In [3]:
params = {}

params['number_epoch'] = 2                               # number of epochs
params['z_dim'] = 10                                     # number of coordinates for SINDy
params['batch_size'] = 8                                # batch size

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_regularization'] = 1e-5

# SINDy parameters
params['sindy_threshold'] = 0.5 
params['poly_order'] = 2
params['include_sine'] = False

### Data processing

In [4]:
# read the videos in random order
file_names = []
for f in listdir('../../Videos/train/'):
    file_names.append(f)

random.shuffle(file_names)

# define transform to tensor and resize to 1080x1920
normalize = transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])    # normalize around mean with sigma (std)
transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((1080, 1920))])

# read data to list and transform to tensor
train_data_tmp = []
train_data = []
train_idxOfNewVideo = []
count = 0
for f in file_names:
    # just for testing (save time)
    if count == 1:
        break
    count += 1
    vidcap = cv2.VideoCapture('../../Videos/train/' + f)
    success,imgR = vidcap.read()
    print('Read:',f)
    while success:
        imgR = cv2.cvtColor(imgR, cv2.COLOR_BGR2RGB)
        imgR_tensor = transform(imgR)
        train_data_tmp.append(imgR_tensor)
        success,imgR = vidcap.read()
        # make a batch
        if len(train_data_tmp) >= params['batch_size']:
            train_data.append(torch.stack(train_data_tmp))
            train_data_tmp = []
    train_idxOfNewVideo.append(len(train_data))
    print(len(train_data), len(train_data[0]), len(train_data[0][0]), len(train_data[0][0][0]), len(train_data[0][0][0][0]))


Read: St_Oct_low.mp4
81 8 3 1080 1920


In [ ]:
print(len(train_data), len(train_data[0]), len(train_data[0][0]), len(train_data[0][0][0]), len(train_data[0][0][0][0]))

# split into validation and training set
validation_data = []
# take 10% of training set batches to validation set
nbr_batch = int(len(train_data)*0.1)
print(nbr_batch)
for i in range(0,nbr_batch):
    choose = random.randint(0, len(train_data)-1)
    element = train_data[choose]
    validation_data.append(element)
    train_data.remove(element)
    print(len(validation_data), len(validation_data[0]), len(validation_data[0][0]), len(validation_data[0][0][0]), len(validation_data[0][0][0][0]))


In [ ]:
print(torch.min(train_data[0]), torch.max(train_data[0]))
print(train_data[0].permute(3,2,1,0).size())

# plot first frame per batch
for i in range(len(train_data)):
    if i%3 == 0:
        plt.figure()
    imgP = train_data[i][0].permute(1,2,0).detach().numpy()
    plt.subplot(1,3, i%3 + 1)
    plt.imshow(imgP)

               

### SINDy autoencoder

In [11]:
# encoder architecture
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()               
        # encoder: N, 3, 1080, 1920
        self.conv1_e = nn.Conv2d(3, 16, 5) # N, 16, 1076, 1916
        self.conv2_e = nn.Conv2d(16, 32, 5) # N, 32, 1072, 1912
        self.pool1_e = nn.MaxPool2d((2,3), stride=(2,3)) # N, 32, 536, 637              -- pool --
        self.conv3_e = nn.Conv2d(32, 64, 8, stride=2, padding=1) # N, 64, 266, 316
        self.conv4_e = nn.Conv2d(64, 96, 7) # N, 96, 260, 310
        self.pool2_e = nn.MaxPool2d(2, stride=2) # N, 96, 130, 155                      -- pool --
        self.conv5_e = nn.Conv2d(96, 128, 11) # N, 128, 120, 145
        self.conv6_e = nn.Conv2d(128, 150, 4, stride=2, padding=1) # N, 150, 60, 72
        self.conv7_e = nn.Conv2d(150, 200, 11) # N, 200, 50, 62
        self.pool3_e = nn.MaxPool2d(2,stride=2) # N, 200, 25, 31                        -- pool --
        self.conv8_e = nn.Conv2d(200, 200, 9, stride=2, padding=1) # N, 200, 10, 13
        self.conv9_e = nn.Conv2d(200, 200, 10) # N, 200, 1, 4
        self.fc1 = nn.Linear(200*4,params['z_dim'])
        # Note: nn.MaxPool2d -> use nn.MaxUnpool2d, or use different kernelsize, stride etc to compensate...
        # Input [-1, +1] -> use nn.Tanh    

    def forward(self, x):
        #print(x.size())
        encoded = F.relu(self.conv1_e(x))
        encoded = F.relu(self.conv2_e(encoded))
        encoded = F.relu(self.pool1_e(encoded))
        encoded = F.relu(self.conv3_e(encoded))
        encoded = F.relu(self.conv4_e(encoded))
        encoded = F.relu(self.pool2_e(encoded))
        encoded = F.relu(self.conv5_e(encoded))
        encoded = F.relu(self.conv6_e(encoded))
        encoded = F.relu(self.conv7_e(encoded))
        encoded = F.relu(self.pool3_e(encoded))
        encoded = F.relu(self.conv8_e(encoded))
        encoded = F.relu(self.conv9_e(encoded))

        #print(encoded.size())
        encoded = encoded.view(-1,200*4)
        encoded = self.fc1(encoded)
        
        return encoded

# decoder architecture
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()               
        # decoder: N, 200, 1, 4
        self.fc2 = nn.Linear(params['z_dim'], 200*4)
        self.convT9_d = nn.ConvTranspose2d(200, 200, 4) # N, 200, 4, 7
        self.convT8_d = nn.ConvTranspose2d(200, 200, 9, stride=2, padding=2, output_padding=(1,0)) # N, 200, 12, 17
        self.convT7_d = nn.ConvTranspose2d(200, 150, 8, stride=4, padding=(0,1), output_padding=(3,0)) # N, 150, 55, 70
        self.convT6_d = nn.ConvTranspose2d(150, 128, 8) # N, 128, 62, 77
        self.convT5_d = nn.ConvTranspose2d(128, 96, 6, stride=(2,3), padding=1) # N, 96, 126, 232
        self.convT4_d = nn.ConvTranspose2d(96, 64, 7) # N, 64, 132, 238
        self.convT3_d = nn.ConvTranspose2d(64, 32, 8, stride=2, padding=(1,2), output_padding=(1,1)) # N, 32, 296, 479
        self.convT2_d = nn.ConvTranspose2d(32, 16, 5, stride=2, padding=1) # N, 16, 539, 959
        self.convT1_d = nn.ConvTranspose2d(16, 3, 5, stride=2, padding=1, output_padding=(1,1)) # N, 3, 1080, 1920

    def forward(self, x):
        decoded = self.fc2(x)
        decoded = decoded.view(-1,200,1,4)
        decoded = F.relu(self.convT9_d(decoded))
        print(decoded.size())
        decoded = F.relu(self.convT8_d(decoded))                                    
        print(decoded.size())
        decoded = F.relu(self.convT7_d(decoded))
        print(decoded.size())
        decoded = F.relu(self.convT6_d(decoded))
        print(decoded.size())
        decoded = F.relu(self.convT5_d(decoded))
        print(decoded.size())                                   
        decoded = F.relu(self.convT4_d(decoded))
        print(decoded.size())
        decoded = F.relu(self.convT3_d(decoded))
        print(decoded.size())
        decoded = F.relu(self.convT2_d(decoded))
        print(decoded.size())
        decoded = torch.sigmoid(self.convT1_d(decoded))
        print(decoded.size())
        
        return decoded

In [12]:
# load model
encoder_path = 'results/Encoder_#epoch_v1.pt'
decoder_path = 'results/Decoder_#epoch_v1.pt'
if os.path.isfile(encoder_path):
    encoder = torch.load(encoder_path)
    print('loaded encoder', encoder_path)
if os.path.isfile(decoder_path):
    encoder = torch.load(decoder_path)
    print('loaded decoder', decoder_path)
else:
    encoder = Encoder()
    decoder = Decoder()
    print('loaded new autoencoder')

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# optimization technique
criterion = nn.MSELoss()
# solution to add both encoder and decoder parameters is unknown?
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-3, weight_decay=1e-5)

# to save network data
network = {}

# training
outputs = []
for epoch in range(params['number_epoch']):
    for batch_id,img in enumerate(train_data):
        encode = encoder(img)                 # encoded
        recon = decoder(encode)               # reconstruction
        # dx, dz is from current batch_id, x, z is from previous batch_id
        if batch_id == 0:
            network['x'] = img
            network['z'] = encode
            #network['x_decode'] = 0
            network['dx'] = img
            network['dz'] = encode
            network['dx_decode'] = recon
        else:
            network['x'] = network['dx']
            network['z'] = network['dz']
            #network['x_decode'] = network['dx_decode']
            network['dx'] = img
            network['dz'] = encode
            network['dx_decode'] = recon
        
        network['rec_loss'] = criterion(recon, img)
        
        # calculate SINDy
        if batch_id >= 1:
            network['dz_predict'] = calculateSindy(network, params)
            network['dx_decode'] = decoder(network['dz_predict'])
            total_loss = calculateLoss(network, params)            # total loss with SINDy
        else:
            total_loss = network['rec_loss']       
        
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
        
        accuracy = 0
        printProgress(epoch, batch_id, accuracy, total_loss)
        if batch_id == 2:
            break
    print('\n')
    outputs.append((epoch, img, recon))
    



loaded new autoencoder
torch.Size([8, 200, 4, 7])
torch.Size([8, 200, 12, 17])
torch.Size([8, 150, 55, 70])
torch.Size([8, 128, 62, 77])
torch.Size([8, 96, 126, 232])
torch.Size([8, 64, 132, 238])
torch.Size([8, 32, 269, 479])
torch.Size([8, 16, 539, 959])
torch.Size([8, 3, 1080, 1920])
Train Epoche 1: 1% [>]	 accuracy: 0.000000, loss: 0.079411torch.Size([8, 200, 4, 7])
torch.Size([8, 200, 12, 17])
torch.Size([8, 150, 55, 70])
torch.Size([8, 128, 62, 77])
torch.Size([8, 96, 126, 232])
torch.Size([8, 64, 132, 238])
torch.Size([8, 32, 269, 479])
torch.Size([8, 16, 539, 959])
torch.Size([8, 3, 1080, 1920])


RuntimeError: expected scalar type Float but found Double

In [ ]:
# example prints
torch.save(model, 'results/Autoencoder_2epoch_v1.pt')


In [ ]:
# testing



### Prediction and plots

In [ ]:
# SINDy prediction --> split up model that I can plug in z
t = np.arange(0,20,.01)
z_sim = sindy.sindy_simulate(network['z'], t, network['Xi'], params['poly_order'], params['include_sine'])

In [ ]:
# plots
print(network['Theta'].shape, network['Xi'].shape)
#print(network['Xi'])

print('reconstruction loss', categorie_loss[0],'\n sindy x loss', categorie_loss[1], 
      '\n sindy z loss', categorie_loss[2], '\n Xi sparsity loss', categorie_loss[3])
print(network['rec_loss'])

for i in range(0, params['number_epoch']):
    plt.figure()
    realImg = outputs[i][1].permute(0,2,3,1).detach().numpy()
    reconImg = outputs[i][2].permute(0,2,3,1).detach().numpy()
    for i, item in enumerate(realImg):
        if i >=4: break
        plt.subplot(2,4,i+1)
        plt.imshow(item)
    for i, item in enumerate(reconImg):
        if i >=4: break
        plt.subplot(2,4,4+i+1)
        plt.imshow(item)    
